In [1]:
import pandas as pd
import numpy as np
import re
import string
import spacy
import gensim
from gensim import corpora 
import nltk
from nltk.corpus import stopwords
import spacy
import en_core_web_md

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from gensim.models.coherencemodel import CoherenceModel

In [2]:
df = pd.read_csv('AmazonReviews.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [3]:
print('Unique users:')
print(len(df.groupby('UserId')))
print('Unique products:')
print(len(df.groupby('ProductId')))

Unique users:
256059
Unique products:
74258


In [4]:
def clean_text(text):
    delect_dict = {special_char: '' for special_char in string.punctuation}
    delect_dict[' '] = ' ' # ??
    table = str.maketrans(delect_dict)  # dict, get keys' ASCII
    text = text.translate(table)  # turn key to its value
    text = text.split()  # tokenize
    text = ' '.join([w for w in text if (not w.isdigit() and len(w)>3)]) # remove digits, word's len>3
    return text.lower()

In [5]:
df.dropna(axis=0, how='any', inplace=True)
df['Text'] = df['Text'].apply(clean_text)
df['Word_count'] = df['Text'].apply(lambda x: len(x.split()))

print(df['Score'].value_counts())
print(len(df))

max_text_len = df['Word_count'].max()
print(max_text_len)

5    363111
4     80655
1     52264
3     42638
2     29743
Name: Score, dtype: int64
568411
2234


In [6]:
condition = (df['Word_count']<100) & (df['Word_count']>=20)
df_short = df[condition]
print(len(df_short))

print(df_short['Score'].value_counts())
print(len(df_short))

373281
5    233648
4     53413
1     36230
3     29080
2     20910
Name: Score, dtype: int64
373281


In [7]:
df_sampled = df_short.groupby('Score').apply(lambda x: x.sample(n=20000)).reset_index(drop=True)
print(len(df_sampled))

100000


In [8]:
def remove_stopwords(text):
    stop_words = stopwords.words('english')
    text = text.split(' ')
    text = ' '.join([w for w in text if w not in stop_words])
    return text

df_sampled['Text'] = df_sampled['Text'].apply(remove_stopwords)

In [9]:
nlp = spacy.load('en_core_web_md', disable = ['parser', 'ner'])

def lemmatization(texts, allowed_postags = ['NOUN', 'ADJ']):
    output = []
    for sentence in texts:
        doc = nlp(sentence)
        output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return output

text_list = df_sampled['Text'].tolist()
tokens = lemmatization(text_list)
print(tokens[1])

['pack', 'teriyaki', 'flavor', 'taste', 'surprised', 'poor', 'quality', 'favorable', 'review', 'batch', 'waste', 'rest', 'time', 'typical', 'link', 'jack', 'link']


In [10]:
dictionary = corpora.Dictionary(tokens)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokens]

In [11]:
LDA = gensim.models.ldamodel.LdaModel
lda_model = LDA(corpus = doc_term_matrix, 
                id2word = dictionary, 
                num_topics = 10, 
                random_state = 100, 
                chunksize = 100, 
                passes = 50,
                iterations = 100
               )

In [12]:
lda_model.print_topics()

[(0,
  '0.041*"quality" + 0.034*"regular" + 0.033*"bar" + 0.028*"health" + 0.028*"bean" + 0.027*"fruit" + 0.025*"enough" + 0.024*"friend" + 0.024*"able" + 0.024*"problem"'),
 (1,
  '0.049*"sugar" + 0.039*"cookie" + 0.031*"ingredient" + 0.027*"product" + 0.027*"tasty" + 0.026*"good" + 0.026*"natural" + 0.024*"calorie" + 0.023*"diet" + 0.020*"coconut"'),
 (2,
  '0.165*"coffee" + 0.063*"chocolate" + 0.045*"flavor" + 0.038*"strong" + 0.037*"good" + 0.028*"taste" + 0.027*"nice" + 0.022*"case" + 0.019*"light" + 0.019*"blend"'),
 (3,
  '0.057*"sweet" + 0.052*"flavor" + 0.037*"wonderful" + 0.029*"sauce" + 0.027*"morning" + 0.022*"variety" + 0.022*"vanilla" + 0.021*"taste" + 0.020*"good" + 0.018*"one"'),
 (4,
  '0.109*"food" + 0.034*"product" + 0.032*"free" + 0.023*"great" + 0.020*"good" + 0.020*"happy" + 0.019*"cat" + 0.019*"large" + 0.018*"brand" + 0.016*"work"'),
 (5,
  '0.057*"dog" + 0.054*"delicious" + 0.048*"milk" + 0.044*"easy" + 0.034*"bag" + 0.030*"green" + 0.021*"small" + 0.020*"tea" 

In [13]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.162501 -0.183060       1        1  23.910231
4     -0.114980 -0.263333       2        1  12.160483
1     -0.129636 -0.034787       3        1  10.839931
2     -0.201969  0.145647       4        1   9.550790
3     -0.151019  0.137794       5        1   9.414426
6      0.213569  0.029883       6        1   8.397408
0      0.167142 -0.158326       7        1   7.661270
5      0.207550 -0.031170       8        1   6.759808
7     -0.033671  0.255869       9        1   5.990258
8      0.205514  0.101484      10        1   5.315398, topic_info=          Term          Freq         Total Category  logprob  loglift
43      coffee  29831.000000  29831.000000  Default  30.0000  30.0000
97        food  25117.000000  25117.000000  Default  29.0000  29.0000
293      great  32430.000000  32430.000000  Default  28.0000  28.0000
158      water   9923.000000   9923.000000  Default  27.0000  27.0000
125  chocolate  11372.000000  11372.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
930      other   1361.373169   2697.082764  Topic10  -4.3036   2.2509
927      color   1022.758301   1706.888672  Topic10  -4.5896   2.4224
255    package    957.676697   3979.940186  Topic10  -4.6553   1.5101
155      small    971.031250   7358.579102  Topic10  -4.6415   0.9093
159       week    897.880310   3870.247803  Topic10  -4.7198   1.4735

[414 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1471      7  0.999683         able
237       2  0.999282      allergy
1175      9  0.999668       almond
238       2  0.999624  alternative
82        8  0.999394      amazing
...     ...       ...          ...
50        1  0.566604         year
50        2  0.289045         year
50        7  0.144324         year
2078      9  0.998523       yogurt
1953      4  0.999565        yummy

[431 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 5, 2, 3, 4, 7, 1, 6, 8, 9])

In [15]:
print('Perplexity:') # the lower the better
# Perp = P(w1w2...wn)^(1/n)
print(lda_model.log_perplexity(doc_term_matrix,total_docs=10000))
coherence_model_lda = CoherenceModel(model = lda_model, texts = tokens, dictionary = dictionary, coherence = 'c_v')
print('Coherence:') # the higher the better
print(coherence_model_lda.get_coherence())

Perplexity:
-8.99045969404222
Coherence:
0.3010274961426592
